# Kernel Generation for image similarity using Wiener Deconvolution 

## Imports

In [1]:
import numpy as np
import cv2
from scipy import signal
from scipy import fftpack
from matplotlib import pyplot as plt
from skimage.metrics import structural_similarity as ssim

plt.rcParams["figure.figsize"] = (20, 20)

## Helper Functions

### `wiener_deconvolution`
Performs wiener deconvolution on a target and input image and outputs a kernal.

In [2]:
def wiener_deconvolution(input_image: np.ndarray, target_image: np.ndarray, noise_level: float = 0.1, kernel_size: int = 15) -> np.ndarray:
    """
    Perform Wiener deconvolution to compute the deblurring kernel.

    Parameters:
    - input_image: np.ndarray - The input (blurry) image.
    - target_image: np.ndarray - The target (sharp) image.
    - noise_level: float - The noise level for regularization (default is 0.1).

    Returns:
    - np.ndarray - The computed deblurring kernel.
    """
   # Compute FFT of input and target images
    input_fft = fftpack.fft2(input_image)
    target_fft = fftpack.fft2(target_image)
    
    # Estimate the filter in frequency domain
    filter_fft = np.conj(input_fft) * target_fft / (np.abs(input_fft)**2 + noise_level**2)
    
    # Convert filter back to spatial domain
    kernel = np.real(fftpack.ifft2(filter_fft))
    
    # Normalize and truncate the kernel
    kernel = kernel / np.sum(kernel)
    kernel = fftpack.fftshift(kernel)
    center = kernel.shape[0] // 2
    kernel = kernel[center - kernel_size // 2:center + kernel_size // 2 + 1,
                    center - kernel_size // 2:center + kernel_size // 2 + 1]
    
    return kernel

### `apply_kernel`
Applies kernel.

In [3]:
def apply_kernel(image: np.ndarray, kernel: np.ndarray) -> np.ndarray:
    return signal.convolve2d(image, kernel, mode='same', boundary='wrap')

### `apply_sharpening`
Applies sharpening to an image based on a given sharpening factor.

In [4]:
def apply_sharpening(image: np.ndarray, sharpening_factor: float):
    """
    Apply sharpening to the image.
    :param image: Input image
    :param sharpening_factor: Factor to control the strength of sharpening (0.0 means no sharpening)
    :return: Sharpened image
    """
    # Define sharpening kernel
    kernel = np.array([[0, -1, 0],
                       [-1, 5 + sharpening_factor, -1],
                       [0, -1, 0]])
    # Apply the kernel to the image
    sharpened_image = cv2.filter2D(image, -1, kernel)
    return sharpened_image

### `adjust_brightness`
Adjusts the brightness of an image based on a given brightness factor.

In [5]:
def adjust_brightness(image: np.ndarray, brightness_factor: float) -> np.ndarray:
    """
    Adjust the brightness of the image.
    :param image: Input image
    :param brightness_factor: Factor by which to increase the brightness (1.0 means no change)
    :return: Brightness adjusted image
    """
    # Convert image to float32 to prevent clipping values during multiplication
    image_float = image.astype(np.float32)
    # Adjust brightness
    brightened_image = image_float * brightness_factor
    # Clip values to the range [0, 255]
    brightened_image = np.clip(brightened_image, 0, 255)
    # Convert back to uint8
    return brightened_image.astype(np.uint8)

In [6]:
def brighten_dark_spots(image, threshold=50, brighten_factor=1.5):
    """
    Brighten only the dark spots of the image.
    :param image: Input grayscale image
    :param threshold: Intensity threshold to identify dark spots
    :param brighten_factor: Factor by which to brighten the dark spots
    :return: Image with brightened dark spots
    """
    # Ensure image is in float32 to prevent clipping values during multiplication
    image_float = image.astype(np.float32)
    
    # Create a mask for dark spots
    dark_spots_mask = image_float < threshold
    
    # Brighten dark spots
    brightened_image = image_float.copy()
    brightened_image[dark_spots_mask] *= brighten_factor
    
    # Clip values to the range [0, 255]
    brightened_image = np.clip(brightened_image, 0, 255)
    
    # Convert back to uint8
    return brightened_image.astype(np.uint8)

## Algo

### Load images in grayscale

In [7]:
def process_image(input_image, target_image, noise_level, kernel_size, sharpening_factor, brightness_factor):
    kernel = wiener_deconvolution(input_image, target_image, noise_level, kernel_size)
    result_image = apply_kernel(input_image, kernel)
    result_image = apply_sharpening(result_image, sharpening_factor)
    result_image = adjust_brightness(result_image, brightness_factor)
    return result_image

def calculate_similarity(img1, img2):
    return ssim(img1, img2)

# Load images
input_image = cv2.imread('Prostate.jpg', cv2.IMREAD_GRAYSCALE)
target_image = cv2.imread('good.jpg', cv2.IMREAD_GRAYSCALE)

# Ensure images are the same size
min_shape = np.minimum(input_image.shape, target_image.shape)
input_image = input_image[:min_shape[0], :min_shape[1]]
target_image = target_image[:min_shape[0], :min_shape[1]]

# Define parameter ranges
noise_levels = [0.05, 0.1, 0.2]
kernel_sizes = [11, 15, 19]
sharpening_factors = [0.3, 0.5, 0.7]
brightness_factors = [1.5, 2.0, 2.5]

best_similarity = -np.inf
best_params = None

# Grid search
for noise_level in noise_levels:
    for kernel_size in kernel_sizes:
        for sharpening_factor in sharpening_factors:
            for brightness_factor in brightness_factors:
                result_image = process_image(input_image, target_image, noise_level, kernel_size, sharpening_factor, brightness_factor)
                similarity = calculate_similarity(result_image, target_image)
                
                if similarity > best_similarity:
                    best_similarity = similarity
                    best_params = (noise_level, kernel_size, sharpening_factor, brightness_factor)

print("Best parameters:")
print(f"Noise level: {best_params[0]}")
print(f"Kernel size: {best_params[1]}")
print(f"Sharpening factor: {best_params[2]}")
print(f"Brightness factor: {best_params[3]}")
print(f"Best similarity: {best_similarity}")

# Generate and save the best result
best_result = process_image(input_image, target_image, *best_params)
cv2.imwrite("best_result.png", best_result)

Best parameters:
Noise level: 0.05
Kernel size: 11
Sharpening factor: 0.7
Brightness factor: 1.5
Best similarity: 0.6894429638847448


True